## Imports e downloads

In [2]:
import nltk
import pandas as pd
import csv
from collections import OrderedDict, defaultdict, Counter
from urllib import request
from nltk import ngrams, FreqDist
from nltk.corpus import floresta


Importing Jupyter notebook from /home/ellen/Projetos/TextMining/src/Header.ipynb


SyntaxError: invalid syntax (<unknown>, line 1)

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('floresta')

[nltk_data] Downloading package punkt to /home/ellen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ellen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package floresta to /home/ellen/nltk_data...
[nltk_data]   Package floresta is already up-to-date!


True

## Coleta de documento

In [3]:
def open_file(filepath):
    file = open(filepath, 'r')

    return file.read()

In [102]:
# URL de Ubirajara no Project Gutenberg
url = 'http://www.gutenberg.org/cache/epub/38496/pg38496.txt'

In [103]:
raw = get_doc(url)

## Pré-processamento

In [104]:
# Tokenização
def tokenize(raw):
    tokens = nltk.word_tokenize(raw.lower())
    del tokens[0:280]
    del tokens[25677:42803]
    
    return tokens

In [105]:
def filter(tokens):
    filtered = [t for t in tokens if t not in stopwords and t.isalpha() and len(t) > 1]
    
    return filtered

In [106]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [107]:
tokens = tokenize(raw)

In [108]:
filtered_tokens = filter(tokens)

In [109]:
text = nltk.Text(tokens)

## Informações básicas sobre o texto

In [110]:
def info():
    print(f"quantidade de palavras: {len(tokens)}")
    print(f"quantidade de palavras após o filtro: {len(filtered_tokens)}")
    print(f"quantidade de palavras únicas: {len(set(tokens))}")
    print(f"quantidade de palavras únicas após o filtro: {len(set(filtered_tokens))}")

info()

quantidade de palavras: 25677
quantidade de palavras após o filtro: 11883
quantidade de palavras únicas: 3607
quantidade de palavras únicas após o filtro: 3255


### Frequência e distribuição de palavras

In [111]:
dist = FreqDist(filtered_tokens)

In [112]:
def most_frequent(tokens):
    sortedToken = sorted(list(set(tokens)), key=lambda token: dist[token], reverse=True)
    frequent_tokens = [(token, dist[token]) for token in sortedToken[:20]]
    
    return frequent_tokens

In [113]:
print("20 palavras mais frequentes:")
most_frequent(filtered_tokens)

20 palavras mais frequentes:


[('guerreiro', 225),
 ('ubirajara', 168),
 ('chefe', 165),
 ('guerreiros', 157),
 ('elle', 145),
 ('grande', 143),
 ('nação', 121),
 ('itaquê', 115),
 ('arací', 97),
 ('virjem', 85),
 ('pojucan', 81),
 ('jurandir', 79),
 ('cabana', 79),
 ('espoza', 76),
 ('arco', 73),
 ('ella', 73),
 ('tocantim', 67),
 ('taba', 66),
 ('jaguarê', 63),
 ('mão', 62)]

In [114]:
print(f"Palavra mais frequente: {dist.max()} - {dist[dist.max()]} vezes")

Palavra mais frequente: guerreiro - 225 vezes


### Análise de contexto

In [115]:
def n_grams(text):
    target_word = dist.max()
    fd = FreqDist(ng
                  for ng in ngrams(text, 5)
                  if target_word in ng)
    for hit in fd:
        print(' '.join(hit))

In [116]:
print("N-grams da palavra de maior frequência \n")
n_grams(text)

N-grams da palavra de maior frequência 

caçador pela gloria do guerreiro
pela gloria do guerreiro .
gloria do guerreiro . para
do guerreiro . para ser
guerreiro . para ser aclamado
. para ser aclamado guerreiro
para ser aclamado guerreiro por
ser aclamado guerreiro por sua
aclamado guerreiro por sua nação
guerreiro por sua nação é
o ferro . nenhum guerreiro
ferro . nenhum guerreiro brandiu
. nenhum guerreiro brandiu jámais
nenhum guerreiro brandiu jámais essa
guerreiro brandiu jámais essa arma
estranjeira dizia que nenhum guerreiro
dizia que nenhum guerreiro jámais
que nenhum guerreiro jámais possuira
nenhum guerreiro jámais possuira a
guerreiro jámais possuira a virjem
dado noticia . -- guerreiro
noticia . -- guerreiro araguaia
. -- guerreiro araguaia ,
-- guerreiro araguaia , pois
guerreiro araguaia , pois vejo
sua lijeireza , joven guerreiro
lijeireza , joven guerreiro ,
, joven guerreiro , elle
joven guerreiro , elle não
guerreiro , elle não te
caçador . feliz o guerreiro
. feliz 

In [117]:
# O método concordance permite ver palavras em um contexto
def context(text):
    target_word = dist.max()
    
    return text.concordance(target_word)

In [118]:
context(text)

Displaying 25 of 25 matches:
a a fama do caçador pela gloria do guerreiro . para ser aclamado guerreiro por 
a do guerreiro . para ser aclamado guerreiro por sua nação é precizo que o jove
a , mais rija que o ferro . nenhum guerreiro brandiu jámais essa arma terrivel 
ta da estranjeira dizia que nenhum guerreiro jámais possuira a virjem formoza .
 os pajés tinham dado noticia . -- guerreiro araguaia , pois vejo pela pena ver
ado te der a sua lijeireza , joven guerreiro , elle não te servirá senão para v
coração feroz do caçador . feliz o guerreiro que te possuir . -- eu sou arací ,
 me terá por espoza . vem comigo , guerreiro araguaia ; excede aos outros no tr
do outro lado da campina assoma um guerreiro . tem na cabeça o canitar das plum
uma franja das mesmas penas . é um guerreiro tocantim . de lonje avistou jaguar
rra ; mas sem quebra da fé póde um guerreiro cansado do longo repouzo oferecer 
 do longo repouzo oferecer a outro guerreiro combate leal . quando o tocantim a
ensajeira d

In [119]:
print("Colocações significantes: \n")
text.collocations()

Colocações significantes: 

grande chefe; dos tocantins; dos araguaias; chefe dos; joven caçador;
nação tocantim; nação araguaia; taba dos; grande arco; seus olhos; dos
guerreiros; das florestas; que elle; seu amor; nesse momento; outra
vez; dos chefes; tres vezes; sua mão; seu braço


In [129]:
text.ContextIndex

AttributeError: 'Text' object has no attribute 'ContextIndex'

## Análise de Emoção

In [120]:
wordList = defaultdict(list)
emotionList = defaultdict(list)

In [121]:
# Importa o léxico e salva em listas
with open('teste.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if int(row['present']) == 1:
            wordList[row['word']].append(row['emotion'])
            emotionList[row['emotion']].append(row['word'])

In [122]:
def generate_count(word_list):
    emoCount = Counter()
    for t in filtered_tokens:
        if len(word_list[t]) > 0:
            emoCount += Counter(word_list[t])
    return emoCount

In [123]:
emotionCounts = generate_count(wordList)
emotionCounts.most_common()

[('positivo', 1529),
 ('negativo', 1220),
 ('medo', 1159),
 ('raiva', 979),
 ('confiança', 723),
 ('tristeza', 532),
 ('alegria', 456),
 ('antecipação', 444),
 ('nojo', 281),
 ('surpresa', 257)]

In [124]:
# Lista só com as palavras presentes no documento
emoList = defaultdict(list)
for t in filtered_tokens:
    for e in emotionList:
        for w in emotionList[e]:
            if w == t:
                emoList[e].append(w)

In [125]:
wordCounts = generate_count(emoList)
wordCounts.most_common(30)

[('amor', 1276),
 ('cabana', 790),
 ('filha', 770),
 ('festa', 765),
 ('guerra', 684),
 ('vencedor', 594),
 ('morte', 550),
 ('alegria', 506),
 ('estrela', 460),
 ('noiva', 460),
 ('guerreiro', 450),
 ('caçador', 440),
 ('paz', 437),
 ('canto', 418),
 ('inimigo', 384),
 ('tristeza', 320),
 ('triunfo', 308),
 ('todos', 280),
 ('feliz', 267),
 ('doce', 253),
 ('robusto', 230),
 ('liberdade', 230),
 ('beleza', 220),
 ('orgulho', 176),
 ('sombrio', 156),
 ('linda', 154),
 ('terrivel', 132),
 ('morrer', 132),
 ('felicidade', 132),
 ('bela', 132)]

In [126]:
for w in wordCounts.most_common(30):
    print(w[0])
    print(wordList[w[0]])
    print('------------------------------------')

amor
['positivo', 'alegria']
------------------------------------
cabana
['positivo', 'tristeza']
------------------------------------
filha
['positivo', 'alegria']
------------------------------------
festa
['positivo', 'antecipação', 'alegria', 'surpresa', 'confiança', 'positivo', 'antecipação', 'alegria', 'surpresa']
------------------------------------
guerra
['negativo', 'medo', 'negativo', 'raiva', 'medo', 'tristeza']
------------------------------------
vencedor
['positivo', 'alegria']
------------------------------------
morte
['negativo', 'raiva', 'antecipação', 'nojo', 'medo', 'tristeza', 'surpresa', 'negativo', 'medo', 'tristeza']
------------------------------------
alegria
['positivo', 'antecipação', 'alegria', 'surpresa', 'confiança']
------------------------------------
estrela
['positivo', 'antecipação', 'alegria', 'confiança']
------------------------------------
noiva
['positivo', 'antecipação', 'alegria', 'confiança', 'positivo', 'antecipação', 'alegria', 'confiança'